# VectorStore-Backed Memory

`VectorStoreRetrieverMemory` stores interactions in a VectorDB and queries the top-K most "salient" interactions every type it is called.

This differs from most of the other Memory classes in that 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain

### Initialize your VectorStore

Depending on the store you choose, this step may look different. Consult the relevant VectorStore documentation for more details.

In [2]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

### Create your the VectorStoreRetrieverMemory

In [3]:
memory = VectorStoreRetrieverMemory(retriever=vectorstore.as_retriever())

# Examples of how the memory object is called
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "Heard of France?"}, {"output": "Undoubtedly"})
memory.save_context({"input": "How about Australia?"}, {"output": "Most certainly"})
memory.save_context({"input": "What about Montreal?"}, {"output": "Of course."})

In [4]:
# Notice the first result discusses France, which the language model deems more semantically relevant
# than the other locations
print(memory.load_memory_variables({"prompt": "Emmanuel Macron"})["history"])

input: Heard of France?
output: Undoubtedly
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: hi
output: whats up


## Using in a chain
Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [5]:
llm = OpenAI() # Can be any valid LLM
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: hi
output: whats up
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great, thanks for asking. How can I help you?"

In [6]:
conversation_with_summary.predict(input="What do you know about Uluru?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
input: hi
output: whats up
input: Hi, what's up?
response:  Hi there! I'm doing great, thanks for asking. How can I help you?
Human: What do you know about Uluru?
AI:

> Finished chain.


'  Uluru is an iconic sandstone monolith located in the Australian Outback. It is considered a sacred site to the Anangu Aboriginal people, who have inhabited the area for thousands of years. It is also a popular tourist destination, with visitors coming from all over the world to appreciate its natural beauty.'

In [7]:
conversation_with_summary.predict(input="Tell me more about UNESCO!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Heard of France?
output: Undoubtedly
input: What do you know about Uluru?
response:   Uluru is an iconic sandstone monolith located in the Australian Outback. It is considered a sacred site to the Anangu Aboriginal people, who have inhabited the area for thousands of years. It is also a popular tourist destination, with visitors coming from all over the world to appreciate its natural beauty.
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
Human: Tell me more about UNESCO!
AI:

> Finished chain.


" UNESCO is a United Nations organization that works to protect and promote the world's cultural and natural heritage. It has designated over 1000 sites around the world as World Heritage Sites, which are recognized for their cultural, historical, and scientific importance. UNESCO also works to promote international cooperation in education, science, and culture."

In [8]:
# We can see here that the buffer is updated
conversation_with_summary.predict(input="Are there UNESCO sites in Canada?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Tell me more about UNESCO!
response:  UNESCO is a United Nations organization that works to protect and promote the world's cultural and natural heritage. It has designated over 1000 sites around the world as World Heritage Sites, which are recognized for their cultural, historical, and scientific importance. UNESCO also works to promote international cooperation in education, science, and culture.
input: What about Montreal?
output: Of course.
input: Heard of France?
output: Undoubtedly
input: What do you know about Uluru?
response:   Uluru is an iconic sandstone monolith located in the Australian Outback. It is considered a sacred site to the Anangu Aboriginal

' Yes, there are 18 UNESCO World Heritage Sites in Canada, including the Historic District of Old Québec, the Rideau Canal, and the Head-Smashed-In Buffalo Jump.'